# sceneflowdataloader.py

In [1]:


import re
import numpy as np
import sys
import chardet 

def readPFM(file):
    file = open(file, 'rb')

    color = None
    width = None
    height = None
    scale = None
    endian = None

    header = file.readline().rstrip()
    encode_type = chardet.detect(header)  
    header = header.decode(encode_type['encoding'])
    if header == 'PF':
        color = True
    elif header == 'Pf':
        color = False
    else:
        raise Exception('Not a PFM file.')

    dim_match = re.match(r'^(\d+)\s(\d+)\s$', file.readline().decode(encode_type['encoding']))
    if dim_match:
        width, height = map(int, dim_match.groups())
    else:
        raise Exception('Malformed PFM header.')

    scale = float(file.readline().rstrip().decode(encode_type['encoding']))
    if scale < 0: # little-endian
        endian = '<'
        scale = -scale
    else:
        endian = '>' # big-endian

    data = np.fromfile(file, endian + 'f')
    shape = (height, width, 3) if color else (height, width)

    data = np.reshape(data, shape)
    data = np.flipud(data)
    return data, scale


# from . import readpfm as rp
def train_test_idxs(datapath):
    folders = os.listdir(f"{datapath}/frames_cleanpass/")
    disparity, segleft, imleft, imright = [], [], [], []
    for folder in folders:
        # disparity.extend(sorted(glob(f"{datapath}/Monkaa/disparity/{folder}/*.pfm")))
        disparity.extend(sorted(glob(f"{datapath}/object_index/{folder}/right/*.pfm")))
        segleft.extend(sorted(glob(f"{datapath}/object_index/{folder}/left/*.pfm"))) 
        imleft.extend(sorted(glob(f"{datapath}/frames_cleanpass/{folder}/left/*.png")))
        imright.extend(sorted(glob(f"{datapath}/frames_cleanpass/{folder}/right/*.png")))

    assert len(disparity) == len(imleft) == len(imright) == len(segleft), f'{len(disparity) , len(imleft) , len(imright) , len(segleft)}'

    idxs = list(range(len(imleft)))
    train_idxs, val_idxs, _, _ = train_test_split(idxs, idxs, test_size=0.2, random_state=42)
    # return train_idxs, val_idxs
    trainimL, valimL = np.array(imleft)[train_idxs], np.array(imleft)[val_idxs]
    trainimR, valimR = np.array(imright)[train_idxs], np.array(imright)[val_idxs]
    trainlabel, valabel = np.array(segleft)[train_idxs],np.array(segleft)[val_idxs]
    traindisp, valdisp = np.array(disparity)[train_idxs], np.array(disparity)[val_idxs] 
    return trainimL, trainimR,trainlabel,traindisp, valimL,  valimR,  valabel,  valdisp

class SceneFlowLoader(Dataset):
    def __init__(self, root, mode = 'train', transform=None, target_transform=None,
                 co_transform=None):
        
        if mode == 'train':
            self.imL_path_list, self.imR_path_list,self.label_path_list,self.disp_path_list,_,_,_,_ = train_test_idxs(root)
        if mode == 'val':
            _,_,_,_,self.imL_path_list, self.imR_path_list,self.label_path_list,self.disp_path_list = train_test_idxs(root)
        print(f"Number of {mode} samples : {len(self.imL_path_list)}")
        self.transform = transform
        self.target_transform = target_transform
        self.co_transform = co_transform
    def __len__(self):
        return len(self.disp_path_list)
    def __getitem__(self, index):
        left  = self.imL_path_list[index]
        right = self.imR_path_list[index]
        disp_L= self.disp_path_list[index]
        seg_L = self.label_path_list[index]
                
        left_img = cv2.imread(left)
        right_img = cv2.imread(right)
        label, _ = readPFM(seg_L)
        disp, scaleL = readPFM(disp_L)
        label =  np.ascontiguousarray(label,dtype=np.float32)[..., np.newaxis]
        
        disp = np.ascontiguousarray(disp,dtype=np.float32)[..., np.newaxis]

        assert (self.transform is not None) and (self.target_transform is not None)
        # print(left_img.shape, right_img.shape, label.shape, disp.shape)
        if self.co_transform is not None:
            inputs, label, disp = self.co_transform([left_img, right_img], label, disp)

        if self.transform is not None:
            imL = self.transform(inputs[0])
            imR = self.transform(inputs[1])

        if self.target_transform is not None:
            label = self.target_transform(label)
            disp = self.target_transform(disp)

        return imL, imR, label, disp



# Train pipeline

In [1]:

import torchvision.transforms as transforms
from dataset.sceneflowdataloader import SceneFlowLoader
import torch

from glob import glob
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
import cv2
import numpy as np
import os
import argparse
import shutil
import time
from tqdm import tqdm

import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torchvision.transforms as transforms
from torchvision.utils import make_grid

import superpixelnet.flow_transforms as flow_transforms
from superpixelnet.models.Spixel_single_layer import SpixelNet1l_bn
from superpixelnet.loss import compute_semantic_pos_loss
import datetime
from tensorboardX import SummaryWriter

from superpixelnet.train_util import *

# psmnet
from  models import *
from models.submodule import disparityregression


best_EPE = -1
n_iter = 0

def imgtensor2np(img):
    return img.permute(1,2,0).detach().cpu().numpy()
def to_device(args, device):
    args_out = []
    for arg in args:
        if isinstance(arg, list):
            arg = [ elem.to(device) for elem in arg ]
        else:
            arg = arg.to(device)
        args_out.append(arg)
    return args_out

class ARG:
    def __init__(self):
        self.dataset = 'SceneFlow'
        self.arch = 'SpixelNet1l_bn'
        # self.data= './data_preprocessing/Data'; self.data= './NYU'
        self.data = "./dataset/Monkaa"
        self.savepath = './checkpoints'
        # self.train_img_height = 256; self.train_img_width= 512 
        self.train_img_height = 128; self.train_img_width= 256
        self.input_img_height = 208;self.input_img_width = 208 
        
        self.workers = 4; self.epochs = 300  *10000
        self.start_epoch = 0; self.epoch_size = 6000; self.batch_size = 1;
        self.solver = 'adam'; self.lr= 0.00005; 
        self.momentum = 0.9; self.beta = 0.999; self.weight_decay=4e-4;self.bias_decay=0
        self.milestones=[200000]; self.additional_step=100000; 
        self.pos_weight = 0.003; self.downsize = 16;
        self.gpu = '0'; self.print_freq = 10; self.record_freq  = 5; self.label_factor=5; self.pretrained = None;
        self.no_date=True
        
        self.maxdisp = 192; self.psmmodel = 'basic'
        self.seed = 1; self.pretrainedpsmnet = None

args = ARG()

# !----- NOTE the current code does not support cpu training -----!
os.environ['CUDA_VISIBLE_DEVICES'] = args.gpu
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    print('Current code does not support CPU training! Sorry about that.')
    exit(1)

# def main():
#     global args, best_EPE, save_path, intrinsic

# ============= savor setting ===================
save_path = '{}_{}_{}epochs{}_b{}_lr{}_posW{}'.format(
    args.arch,
    args.solver,
    args.epochs,
    '_epochSize'+str(args.epoch_size) if args.epoch_size > 0 else '',
    args.batch_size,
    args.lr,
    args.pos_weight,
)
if not args.no_date:
    timestamp = datetime.datetime.now().strftime("%y_%m_%d_%H_%M")
else:
    timestamp = ''
save_path = os.path.abspath(args.savepath) + '/' + os.path.join(args.dataset, save_path  +  '_' + timestamp )

# ==========  Data loading code ==============
input_transform = transforms.Compose([
    flow_transforms.ArrayToTensor(),
    transforms.Normalize(mean=[0,0,0], std=[255,255,255]),
    transforms.Normalize(mean=[0.411,0.432,0.45], std=[1,1,1])
])

val_input_transform = transforms.Compose([
    flow_transforms.ArrayToTensor(),
    transforms.Normalize(mean=[0, 0, 0], std=[255, 255, 255]),
    transforms.Normalize(mean=[0.411, 0.432, 0.45], std=[1, 1, 1])
])

target_transform = transforms.Compose([
    flow_transforms.ArrayToTensor(),
])

co_transform = flow_transforms.Compose([
        flow_transforms.RandomCrop((args.train_img_height ,args.train_img_width))
    ])

N_CLASSES = 50
trainset = SceneFlowLoader(args.data, mode='train', transform=input_transform, target_transform=target_transform, co_transform=co_transform)
valset = SceneFlowLoader(args.data, mode='val', transform=val_input_transform, target_transform=target_transform, co_transform=co_transform)

train_loader = torch.utils.data.DataLoader(
    trainset, batch_size=args.batch_size,
    num_workers=args.workers, pin_memory=True, shuffle=True, drop_last=True)

val_loader = torch.utils.data.DataLoader(
    valset, batch_size=args.batch_size,
    num_workers=args.workers, pin_memory=True, shuffle=False, drop_last=True)




Total classes 47.
Number of train samples : 6931
Total classes 47.
Number of val samples : 1733


In [2]:
# import models.stackhourglass.PSMNet as stackhourglass
torch.manual_seed(args.seed)
if device.type  == 'cuda':
    torch.cuda.manual_seed(args.seed)


# ============== create model ====================
if args.pretrained:
    network_data = torch.load(args.pretrained)
    args.arch = network_data['arch']
    print("=> using pre-trained model '{}'".format(args.arch))
else:
    network_data = None
    print("=> creating model '{}'".format(args.arch))

model = SpixelNet1l_bn( data = network_data).cuda()
model = torch.nn.DataParallel(model).cuda()

if args.psmmodel == 'stackhourglass':
    psmnet = stackhourglass(args.maxdisp, slicmode=False)
elif args.psmmodel == 'basic':
    psmnet = basic(args.maxdisp)
else:
    print('no model')
psmnet = torch.nn.DataParallel(psmnet)
psmnet.to(device)

if args.pretrainedpsmnet is not None:
    print('Load pretrained model')
    pretrain_dict = torch.load(args.pretrainedpsmnet)
    psmnet.load_state_dict(pretrain_dict['state_dict'])

print('Number of PSM model parameters: {}'.format(sum([p.data.nelement() for p in psmnet.parameters()])))


cudnn.benchmark = True

#=========== creat optimizer, we use adam by default ==================
assert(args.solver in ['adam', 'sgd'])
print('=> setting {} solver'.format(args.solver))

param_groups = [{'params': model.module.bias_parameters(), 'weight_decay': args.bias_decay},
                {'params': model.module.weight_parameters(), 'weight_decay': args.weight_decay},
                ]
param_groups.extend([{'params': psmnet.module.bias_parameters(), 'weight_decay': args.weight_decay},
                    {'params': psmnet.module.weight_parameters(), 'weight_decay': args.weight_decay}
                    ])
if args.solver == 'adam':
    optimizer = torch.optim.Adam(param_groups, args.lr,
                                    betas=(args.momentum, args.beta))
elif args.solver == 'sgd':
    optimizer = torch.optim.SGD(param_groups, args.lr,
                                momentum=args.momentum)

# for continues training
if args.pretrained and ('dataset' in network_data):
    if args.pretrained and args.dataset == network_data['dataset'] :
        optimizer.load_state_dict(network_data['optimizer'])
        best_EPE = network_data['best_EPE']
        args.start_epoch = network_data['epoch']
        save_path = os.path.dirname(args.pretrained)



print('=> will save everything to {}'.format(save_path))
if not os.path.exists(save_path):
    os.makedirs(save_path)

train_writer = SummaryWriter(os.path.join(save_path, 'train'))
val_writer = SummaryWriter(os.path.join(save_path, 'val'))

# spixelID: superpixel ID for visualization,
# XY_feat: the coordinate feature for position loss term
spixelID, XY_feat_stack = init_spixel_grid(args)
val_spixelID,  val_XY_feat_stack = init_spixel_grid(args, b_train=False)


=> creating model 'SpixelNet1l_bn'
Number of PSM model parameters: 3287360
=> setting adam solver
=> will save everything to /home/gokul/courses/CMSC828I/dispTask/checkpoints/SceneFlow/SpixelNet1l_bn_adam_3000000epochs_epochSize6000_b1_lr5e-05_posW0.003_


In [35]:
def computeOutput(cost, Ql, S):
    upsampled_cost = upfeat(cost, Ql, S, S)
    assert upsampled_cost.size()[2] == Ql.size()[2]
    assert upsampled_cost.size()[3] == Ql.size()[3]
    upsampled_cost = F.upsample(upsampled_cost.unsqueeze(1), [args.maxdisp,Ql.size()[2],Ql.size()[3]], mode='trilinear').squeeze(1)
    output = F.softmax(upsampled_cost)
    output = disparityregression(args.maxdisp)(output)
    return output

def train(train_loader, model, optimizer, epoch, train_writer, init_spixl_map_idx, XY_feat_stack):
    global n_iter, args, intrinsic

    batch_time = AverageMeter()
    data_time = AverageMeter()

    total_loss = AverageMeter()
    losses_slic = AverageMeter()
    losses_psm = AverageMeter()

    epoch_size =  len(train_loader) if args.epoch_size == 0 else min(len(train_loader), args.epoch_size)

    # switch to train mode
    model.train()
    psmnet.train()
    end = time.time()
    iteration = 0

    for i, sample in enumerate(train_loader):
        imL, imR, label, labelR, disp_true = to_device(sample, device)
        iteration = i + epoch * epoch_size

        # ========== adjust lr if necessary  ===============
        if (iteration + 1) in args.milestones:
            state_dict = optimizer.state_dict()
            for param_group in state_dict['param_groups']:
                param_group['lr'] = args.lr * ((0.5) ** (args.milestones.index(iteration + 1) + 1))
            optimizer.load_state_dict(state_dict)

        # ========== complete data loading ================
        label_1hot = label2one_hot_torch(label, C=N_CLASSES) # set C=50 as SSN does
        LABXY_feat_tensor = build_LABXY_feat(label_1hot, XY_feat_stack)  # B* (50+2 )* H * W
        label_1hotR = label2one_hot_torch(labelR, C=N_CLASSES) # set C=50 as SSN does
        LABXY_feat_tensorR = build_LABXY_feat(label_1hotR, XY_feat_stack)  # B* (50+2 )* H * W

        torch.cuda.synchronize()
        data_time.update(time.time() - end)


        # ========== predict association map ============
        Ql = model(imL)
        Qr = model(imR)
        # ========== compute slic loss ============
        slic_lossL, loss_sem, loss_pos = compute_semantic_pos_loss( Ql, LABXY_feat_tensor,
                                                                pos_weight= args.pos_weight, kernel_size=args.downsize)
        slic_lossR, _, _ = compute_semantic_pos_loss( Qr, LABXY_feat_tensorR,
                                                                pos_weight= args.pos_weight, kernel_size=args.downsize)
        
        slic_loss = 0.5* (slic_lossL + slic_lossR)

        # ========== compute disparity map ============
        mask = disp_true < args.maxdisp
        mask.detach_()
        #----
        S = 4; m = 0.003
        pooled_imL = poolfeat(imL, Ql.clone(), S, S)
        pooled_imR = poolfeat(imR, Qr.clone(), S, S)

        if args.psmmodel == 'stackhourglass':
            cost1, cost2, cost3 = psmnet(pooled_imL, pooled_imR)
            # output1 = torch.squeeze(output1,1); output2 = torch.squeeze(output2,1); output3 = torch.squeeze(output3,1)
            # psmloss = 0.5*F.smooth_l1_loss(output1[mask], disp_true[mask], size_average=True) + \
            #     0.7*F.smooth_l1_loss(output2[mask], disp_true[mask], size_average=True) + \
            #         F.smooth_l1_loss(output3[mask], disp_true[mask], size_average=True) 
        elif args.psmmodel == 'basic':
            cost = psmnet(pooled_imL,pooled_imR)
            output  = computeOutput(cost, Ql, S)
            # output = torch.squeeze(output,1)
            psmloss = F.smooth_l1_loss(output[mask], disp_true[mask], size_average=True)

        loss = psmloss + slic_loss

        # ========= back propagate ===============

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # ========  measure batch time ===========
        torch.cuda.synchronize()
        batch_time.update(time.time() - end)
        end = time.time()

        # =========== record and display the loss ===========
        # record loss and EPE
        total_loss.update(loss.item(), imL.size(0))
        losses_slic.update(slic_loss.item(), imL.size(0))
        losses_psm.update(psmloss.item(), imL.size(0))



        if i % args.print_freq == 0:
            print('train Epoch: [{0}][{1}/{2}]\t Time {3}\t Data {4}\t Total_loss {5}\t Loss_sem {6}\t Loss_pos {7}\t'
                    .format(epoch, i, epoch_size, batch_time, data_time, total_loss, losses_slic, losses_psm))

            train_writer.add_scalar('Train_loss', slic_loss.item(), i + epoch*epoch_size)
            train_writer.add_scalar('learning rate',optimizer.param_groups[0]['lr'], i + epoch * epoch_size)

        n_iter += 1
        if i >= epoch_size:
            break

        if (iteration) >= (args.milestones[-1] + args.additional_step):
            break
        
        # =========== write information to tensorboard ===========
        if epoch % args.record_freq == 0:
            train_writer.add_scalar('Train_loss_epoch', slic_loss.item(),  epoch )
            train_writer.add_scalar('loss_sem',  loss_sem.item(),  epoch )
            train_writer.add_scalar('loss_pos',  loss_pos.item(), epoch)

            #save image
            mean_values = torch.tensor([0.411, 0.432, 0.45], dtype=imL.dtype).view(3, 1, 1)
            input_l_save = (make_grid((input + mean_values).clamp(0, 1), nrow=args.batch_size))
            label_save = make_grid(args.label_factor * label)

            train_writer.add_image('Input', input_l_save, epoch)
            train_writer.add_image('label', label_save, epoch)

            # init_spixl_map_idx = spixelID
            curr_spixl_map = update_spixl_map(init_spixl_map_idx,Ql)
            spixel_lab_save = make_grid(curr_spixl_map, nrow=args.batch_size)[0, :, :]
            spixel_viz, _ = get_spixel_image(input_l_save, spixel_lab_save)
            train_writer.add_image('Spixel viz', spixel_viz, epoch)

            # save associ map,  --- for debug only
            _, prob_idx = torch.max(Ql, dim=1, keepdim=True)
            prob_map_save = make_grid(assign2uint8(prob_idx))
            train_writer.add_image('assigment idx', prob_map_save, epoch)

            train_writer.add_image('disp map', imgtensor2np(output[0]) , epoch, dataformats='HWC')
            train_writer.add_image('disp map GT', imgtensor2np(disp_true[0]) , epoch,dataformats='HWC')
            print('==> write train step %dth to tensorboard' % i)

        return total_loss.avg, losses_psm.avg, iteration

def validate(val_loader, networks, epoch, val_writer, init_spixl_map_idx, XY_feat_stack):
    global n_iter,   args,    intrinsic
    batch_time = AverageMeter()
    data_time = AverageMeter()

    total_loss = AverageMeter()
    losses_psm = AverageMeter()
    losses_slic = AverageMeter()

    # set the validation epoch-size, we only randomly val. 400 batches during training to save time
    epoch_size = min(len(val_loader), 400)

    model, psmnet = networks
    # switch to eval mode
    model.eval()
    psmnet.eval()
    end = time.time()

    with torch.no_grad():
        for i, sample in enumerate(val_loader):
            imL, imR, label, labelR, disp_true = to_device(sample, device)


            # ========== complete data loading ================
            label_1hot = label2one_hot_torch(label, C=N_CLASSES) # set C=50 as SSN does
            LABXY_feat_tensor = build_LABXY_feat(label_1hot, XY_feat_stack)  # B* (50+2 )* H * W
            label_1hotR = label2one_hot_torch(labelR, C=N_CLASSES) # set C=50 as SSN does
            LABXY_feat_tensorR = build_LABXY_feat(label_1hotR, XY_feat_stack)  # B* (50+2 )* H * W

            torch.cuda.synchronize()
            data_time.update(time.time() - end)


            # ========== predict association map ============
            Ql = model(imL)
            Qr = model(imR)
            # ========== compute slic loss ============
            slic_lossL, loss_sem, loss_pos = compute_semantic_pos_loss( Ql, LABXY_feat_tensor,
                                                                    pos_weight= args.pos_weight, kernel_size=args.downsize)
            slic_lossR, _, _ = compute_semantic_pos_loss( Qr, LABXY_feat_tensorR,
                                                                    pos_weight= args.pos_weight, kernel_size=args.downsize)
            
            slic_loss = 0.5* (slic_lossL + slic_lossR)

            # ========== compute disparity map ============
            mask = disp_true < args.maxdisp
            mask.detach_()
            #----
            S = 4; m = 0.003
            pooled_imL = poolfeat(imL, Ql.clone(), S, S)
            pooled_imR = poolfeat(imR, Qr.clone(), S, S)

            cost = psmnet(pooled_imL,pooled_imR)
            output  = computeOutput(cost, Ql, S)
            # output = torch.squeeze(output,1)
            psmloss = F.smooth_l1_loss(output[mask], disp_true[mask], size_average=True)

            loss = psmloss + slic_loss

            # measure elapsed time
            torch.cuda.synchronize()
            batch_time.update(time.time() - end)
            end = time.time()

            # record loss and EPE
            total_loss.update(loss.item(), imL.size(0))
            losses_psm.update(psmloss.item(), imL.size(0))
            losses_slic.update(slic_loss.item(), imL.size(0))

            if i % args.print_freq == 0:
                print('val Epoch: [{0}][{1}/{2}]\t Time {3}\t Data {4}\t Total_loss {5}\t Loss_sem {6}\t Loss_pos {7}\t'
                    .format(epoch, i, epoch_size, batch_time, data_time, total_loss, losses_psm, losses_slic))

            if i >= epoch_size:
                break

    # =============  write result to tensorboard ======================
    if epoch % args.record_freq == 0:
        val_writer.add_scalar('val_loss_epoch', loss.item(), epoch)
        val_writer.add_scalar('val_loss_sem', psmloss.item(), epoch)
        val_writer.add_scalar('val_loss_slic', slic_loss.item(), epoch)

        mean_values = torch.tensor([0.411, 0.432, 0.45], dtype=imL.dtype).view(3, 1, 1)
        input_l_save = (make_grid((input + mean_values).clamp(0, 1), nrow=args.batch_size))


        curr_spixl_map = update_spixl_map(init_spixl_map_idx, output)
        spixel_lab_save = make_grid(curr_spixl_map, nrow=args.batch_size)[0, :, :]
        spixel_viz, _ = get_spixel_image(input_l_save, spixel_lab_save)

        label_save = make_grid(args.label_factor * label)

        val_writer.add_image('Input', input_l_save, epoch)
        val_writer.add_image('label', label_save, epoch)
        val_writer.add_image('Spixel viz', spixel_viz, epoch)

        val_writer.add_image('disp map', imgtensor2np(output[0]) , epoch, dataformats='HWC')
        val_writer.add_image('disp map GT', imgtensor2np(disp_true[0]) , epoch,dataformats='HWC')

        # --- for debug
        #     _, prob_idx = torch.max(assign, dim=1, keepdim=True)
        #     prob_map_save = make_grid(assign2uint8(prob_idx))
        #     val_writer.add_image('assigment idx level %d' % j, prob_map_save, epoch)

        print('==> write val step %dth to tensorboard' % i)

    return total_loss.avg, losses_psm.avg


def save_checkpoint(state, is_best, filename='checkpoint.tar'):
    torch.save(state, os.path.join(save_path,filename))
    if is_best:
        shutil.copyfile(os.path.join(save_path,filename), os.path.join(save_path,'model_best.tar'))


#===================================================================================
#=============================== Train routine =====================================
#===================================================================================

for epoch in range(args.start_epoch, args.epochs): # epochs chosen to be 3e6 because we want to end training based on iterations - 300k
    # train for one epoch
    train_avg_slic, train_avg_sem, iteration = train(train_loader, model, optimizer, epoch,
                                                        train_writer, spixelID, XY_feat_stack )
    if epoch % args.record_freq == 0:
        train_writer.add_scalar('Mean avg_slic', train_avg_slic, epoch)

    # evaluate on validation set and save the module( and choose the best)
    with torch.no_grad():
        avg_slic, avg_sem  = validate(val_loader, model, epoch, val_writer, val_spixelID, val_XY_feat_stack)
        if epoch % args.record_freq == 0:
            val_writer.add_scalar('Mean avg_slic', avg_slic, epoch)

    rec_dict = {
            'epoch': epoch + 1,
            'arch': args.arch,
            'state_dict': model.module.state_dict(),
            'best_EPE': best_EPE,
            'optimizer': optimizer.state_dict(),
            'dataset': args.dataset
        }

    if (iteration) >= (args.milestones[-1] + args.additional_step):
        save_checkpoint(rec_dict, is_best =False, filename='%d_step.tar' % iteration)
        print("Train finished!")
        break

    if best_EPE < 0:
        best_EPE = avg_sem
    is_best = avg_sem < best_EPE
    best_EPE = min(avg_sem, best_EPE)
    save_checkpoint(rec_dict, is_best)

torch.Size([1, 1, 128, 256])

/home/gokul/.local/lib/python3.6/site-packages/torch/nn/functional.py:3487: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")
/home/gokul/.local/lib/python3.6/site-packages/torch/nn/functional.py:3613: UserWarning: Default upsampling behavior when mode=trilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


In [28]:
upsampled_cost.shape

torch.Size([1, 192, 128, 256])

torch.Size([1, 192, 1, 1])

(torch.Size([1, 1, 192, 128, 256]), torch.Size([1, 48, 128, 256]))

/home/gokul/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  after removing the cwd from sys.path.


In [30]:
pred.shape

torch.Size([1, 1, 128, 256])

# PSM NET

In [11]:

import torchvision.transforms as transforms
from dataset.sceneflowdataloader import SceneFlowLoader
import torch

from glob import glob
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
import cv2
import numpy as np
import os
import argparse
import shutil
import time
from tqdm import tqdm

import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torchvision.transforms as transforms
from torchvision.utils import make_grid

import superpixelnet.flow_transforms as flow_transforms
from superpixelnet.models.Spixel_single_layer import SpixelNet1l_bn
from superpixelnet.loss import compute_semantic_pos_loss
import datetime
from tensorboardX import SummaryWriter

from superpixelnet.train_util import poolfeat, upfeat, update_spixl_map

best_EPE = -1
n_iter = 0

def imgtensor2np(img):
    return img.permute(1,2,0).detach().cpu().numpy()
def to_device(args, device):
    args_out = []
    for arg in args:
        if isinstance(arg, list):
            arg = [ elem.to(device) for elem in arg ]
        else:
            arg = arg.to(device)
        args_out.append(arg)
    return args_out

class ARG:
    def __init__(self):
        self.dataset = 'SceneFlow'
        self.arch = 'SpixelNet1l_bn'
        # self.data= './data_preprocessing/Data'; self.data= './NYU'
        self.data = "./dataset/Monkaa"
        self.savepath = './checkpoints'
        self.train_img_height = 256; self.train_img_width= 512 
        # self.train_img_height = 64; self.train_img_width= 128
        self.input_img_height = 208;self.input_img_width = 208 
        
        self.workers = 4; self.epochs = 300  *10000
        self.start_epoch = 0; self.epoch_size = 6000; self.batch_size = 1;
        self.solver = 'adam'; self.lr= 0.00005; 
        self.momentum = 0.9; self.beta = 0.999; self.weight_decay=4e-4;self.bias_decay=0
        self.milestones=[200000]; self.additional_step=100000; 
        self.pos_weight = 0.003; self.downsize = 16;
        self.gpu = '0'; self.print_freq = 10; self.record_freq  = 5; self.label_factor=5; self.pretrained = None;
        self.no_date=True
        
        self.maxdisp = 192; self.psmmodel = 'basic'
        self.seed = 1; self.pretrainedpsmnet = None

args = ARG()

# !----- NOTE the current code does not support cpu training -----!
os.environ['CUDA_VISIBLE_DEVICES'] = args.gpu
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    print('Current code does not support CPU training! Sorry about that.')
    exit(1)

# def main():
#     global args, best_EPE, save_path, intrinsic

# ============= savor setting ===================
save_path = '{}_{}_{}epochs{}_b{}_lr{}_posW{}'.format(
    args.arch,
    args.solver,
    args.epochs,
    '_epochSize'+str(args.epoch_size) if args.epoch_size > 0 else '',
    args.batch_size,
    args.lr,
    args.pos_weight,
)
if not args.no_date:
    timestamp = datetime.datetime.now().strftime("%y_%m_%d_%H_%M")
else:
    timestamp = ''
save_path = os.path.abspath(args.savepath) + '/' + os.path.join(args.dataset, save_path  +  '_' + timestamp )

# ==========  Data loading code ==============
input_transform = transforms.Compose([
    flow_transforms.ArrayToTensor(),
    transforms.Normalize(mean=[0,0,0], std=[255,255,255]),
    transforms.Normalize(mean=[0.411,0.432,0.45], std=[1,1,1])
])

val_input_transform = transforms.Compose([
    flow_transforms.ArrayToTensor(),
    transforms.Normalize(mean=[0, 0, 0], std=[255, 255, 255]),
    transforms.Normalize(mean=[0.411, 0.432, 0.45], std=[1, 1, 1])
])

target_transform = transforms.Compose([
    flow_transforms.ArrayToTensor(),
])

co_transform = flow_transforms.Compose([
        flow_transforms.RandomCrop((args.train_img_height ,args.train_img_width))
    ])

N_CLASSES = 50
trainset = SceneFlowLoader(args.data, mode='train', transform=input_transform, target_transform=target_transform, co_transform=co_transform)
valset = SceneFlowLoader(args.data, mode='val', transform=val_input_transform, target_transform=target_transform, co_transform=co_transform)

train_loader = torch.utils.data.DataLoader(
    trainset, batch_size=args.batch_size,
    num_workers=args.workers, pin_memory=True, shuffle=True, drop_last=True)

val_loader = torch.utils.data.DataLoader(
    valset, batch_size=args.batch_size,
    num_workers=args.workers, pin_memory=True, shuffle=False, drop_last=True)




Total classes 47.
Number of train samples : 6931
Total classes 47.
Number of val samples : 1733


Number of model parameters: 5224768


In [13]:
for i, sample in enumerate(train_loader):
    imL, imR, label, labelR, disp_true = to_device(sample, device)
    break    

In [14]:



    # return loss.data


feature extractor oskip: torch.Size([1, 128, 64, 128])
feature extractor obranch1: torch.Size([1, 32, 64, 128])
feature extractor obranch2: torch.Size([1, 32, 64, 128])
feature extractor obranch3: torch.Size([1, 32, 64, 128])
feature extractor obranch4: torch.Size([1, 32, 64, 128])
feature extractor output_feature: torch.Size([1, 320, 64, 128])
feature extractor oskip: torch.Size([1, 128, 64, 128])
feature extractor obranch1: torch.Size([1, 32, 64, 128])
feature extractor obranch2: torch.Size([1, 32, 64, 128])
feature extractor obranch3: torch.Size([1, 32, 64, 128])
feature extractor obranch4: torch.Size([1, 32, 64, 128])
feature extractor output_feature: torch.Size([1, 320, 64, 128])
feature shape , torch.Size([1, 32, 64, 128])
cost shape , torch.Size([1, 64, 48, 64, 128])
cost0 shape , torch.Size([1, 32, 48, 64, 128])
out1 shape , torch.Size([1, 32, 48, 64, 128])
out2 shape , torch.Size([1, 32, 48, 64, 128])
out3 shape , torch.Size([1, 32, 48, 64, 128])
cost shapes 1, 2, 3:  torch.Si

/home/gokul/.local/lib/python3.6/site-packages/torch/nn/functional.py:3487: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")
/home/gokul/.local/lib/python3.6/site-packages/torch/nn/functional.py:3613: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)
/home/gokul/.local/lib/python3.6/site-packages/torch/nn/functional.py:3613: UserWarning: Default upsampling behavior when mode=trilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


pred shapes 1, 2, 3:  torch.Size([1, 1, 256, 512]) torch.Size([1, 1, 256, 512]) torch.Size([1, 192, 256, 512])


/home/gokul/.local/lib/python3.6/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


torch.Size([1, 64, 128])

In [20]:
torch.sum(torch.isnan(disp_true))

tensor(0, device='cuda:0')

In [9]:
output[mask]

tensor([], device='cuda:0', grad_fn=<IndexBackward>)

In [8]:
torch.isnan(output[mask]), torch.isnan(disp_true[mask])

(tensor([], device='cuda:0', dtype=torch.bool),
 tensor([], device='cuda:0', dtype=torch.bool))

In [6]:
output.shape

torch.Size([1, 64, 128])

In [7]:
mask.shape

torch.Size([1, 1, 64, 128])